# PREVISÃO DO TEMPO EM PYTHON COM API DA OPEN WEATHER MAP + RESULTADO EM AUDIO + MÉDIA DOS 5 DIAS

In [ ]:
import requests
import json
import pyttsx3
from datetime import datetime
from collections import defaultdict

city = input("Digite a cidade para a qual deseja obter informações meteorológicas: ")

# Insira sua chave de API aqui - se não tiver gere aqui https://openweathermap.org/
api_key = "DIGITE SUA CHAVE API DO OPEN WEATHER MAP"
forecast_url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric&lang=pt_br"
response = requests.get(forecast_url)

# Verifica se a requisição foi bem sucedida
if response.status_code == 200:
    # Se a solicitação foi bem-sucedida, analise os dados JSON
    forecast_data = json.loads(response.text)
    
    # Verifica se a cidade foi encontrada
    if forecast_data["cod"] == "200":
        forecast_list = forecast_data["list"]
        
        # Dicionário para armazenar as temperaturas e descrições por dia
        daily_temps = defaultdict(list)
        daily_weather = {}

        # Iterar sobre a lista de previsões e agrupar por dia
        for forecast in forecast_list:
            date_time = datetime.strptime(forecast["dt_txt"], "%Y-%m-%d %H:%M:%S")
            day_month = date_time.strftime("%d/%m")
            temperature = forecast["main"]["temp"]
            weather_description = forecast["weather"][0]["description"]
            
            daily_temps[day_month].append(temperature)
            if day_month not in daily_weather:
                daily_weather[day_month] = weather_description
        
        print(f"Previsão do tempo para {city} nos próximos 5 dias:")
        engine = pyttsx3.init()
        engine.say(f"Previsão do tempo para {city} nos próximos 5 dias.")
        
        # Calcular e exibir a mínima, máxima e descrição do tempo por dia
        for day, temps in list(daily_temps.items())[:5]:  # Considerar apenas os próximos 5 dias
            max_temp = max(temps)
            min_temp = min(temps)
            weather_description = daily_weather[day]
            print(f"{day}: Mínima {min_temp:.1f}°C, Máxima {max_temp:.1f}°C, Tempo: {weather_description}")
            engine.say(f"No dia {day}, a mínima de temperatura será de {min_temp:.1f} graus Celsius, "
                       f"a máxima será de {max_temp:.1f} graus Celsius, com tempo {weather_description}.")
        
        engine.runAndWait()
    else:
        print(f"Não foi possível encontrar a cidade: {city}. Verifique se o nome está correto.")
        engine = pyttsx3.init()
        engine.say(f"Não foi possível encontrar a cidade: {city}. Verifique se o nome está correto.")
        engine.runAndWait()
else:
    # Se a solicitação não for bem-sucedida, imprima uma mensagem de erro detalhada
    print(f"Ocorreu um erro ao obter as informações meteorológicas. Código de status: {response.status_code}")
    engine = pyttsx3.init()
    engine.say(f"Ocorreu um erro ao obter as informações meteorológicas. Código de status: {response.status_code}")
    engine.runAndWait()

    